In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!cp /usr/bin/chromium-browser /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,758 kB]
Hit:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:15 http://ppa.launchp

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml

In [4]:
!pip install selenium

In [0]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

**ในกรณีที่ไม่ได้ใช้ url ที่เป็นจังหวัดหรือเขต จะไม่สามารถดึงข้อมูลผ่าน requests ได้โดยตรง เนื่องจาก element ต่างๆ ในเว็บจะยังไม่ถูก Loaded ขึ้นมาจึงเลือกใช้ Selenium แทน requests เพื่อรองรับกรณีนี้**


In [0]:
from bs4 import BeautifulSoup

In [0]:
def getHTMLContent(url): 
  options = webdriver.ChromeOptions()
  options.add_argument('--no-sandbox')
  options.add_argument('--headless')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--remote-debugging-port=9222')
  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url)
  timeout = 120
  page = ''
  try:
      WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CLASS_NAME, "aqi-card-info-number")))
      page = driver.page_source
      html = BeautifulSoup(page, 'html.parser')
  except TimeoutException:
      driver.quit()
      html = None
  return html

In [0]:
def extractInfo(html):
  if html.find('div', class_='page-title') is None:
    return None
  name = html.find('div', class_='page-title').h1.b.text
  status = html.find('div', class_='aqi-card-info-status').find('span', class_='status-text').text 
  aqi = html.find('div', class_='aqi-card-info-number').find('span', class_='aqi').text
  aqi = aqi[0:2]
  return name, status, aqi

In [9]:
urls = {
  'Home': 'https://www.airvisual.com/thailand/bangkok/yan-nawa',
  'Office': 'https://www.airvisual.com/thailand/bangkok/din-daeng-rd-din-daeng'
}
for key, url in urls.items():
  html = getHTMLContent(url)
  name, status, aqi = extractInfo(html)
  print("Location: {} ({}) | {} US AQI ({}) ".format(key, name, aqi, status))

Location: Home (Yan Nawa) | 57 US AQI (Moderate) 
Location: Office (Din Daeng Rd., Din Daeng) | 78 US AQI (Moderate) 
